# 🏈 Automatic Draft Order Management

Streamlined draft interface that automatically handles team assignments based on draft position order.

## Features:
- ✅ **Auto team assignment** - No need to select teams manually
- ✅ **Draft position management** - Set draft positions once
- ✅ **Snake draft automation** - Handles pick order automatically
- ✅ **One-click drafting** - Just select player and draft
- ✅ **Real-time tracking** - Live updates for all teams
- ✅ **Position-based filtering** - Quick player searches

In [1]:
# Required imports
import pandas as pd
import numpy as np
import yaml
import json
import pickle
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("🏈 Auto-Draft Order Management System")
print("🚀 Loading data and initializing automated draft system...")

🏈 Auto-Draft Order Management System
🚀 Loading data and initializing automated draft system...


In [2]:
# Load configuration and data
with open('config/league-config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load draft cheat sheet (our main player rankings)
cheat_sheet = pd.read_csv('draft_cheat_sheet.csv')

print(f"✅ Loaded {len(cheat_sheet)} players")
print(f"✅ League: {config['league_name']} ({config['basic_settings']['teams']} teams)")
print(f"✅ Draft: {config['draft']['type']} draft")

✅ Loaded 50 players
✅ League: Fantasy 24-25 (14 teams)
✅ Draft: Snake draft


In [3]:
class AutoDraftTracker:
    """Automated draft tracking with position-based team assignments"""
    
    def __init__(self, config, cheat_sheet):
        self.config = config
        self.num_teams = config['basic_settings']['teams']
        self.roster_size = config['roster']['total_size']
        self.team_names = config['team_names']
        
        # Draft position assignments (can be customized)
        self.draft_positions = {team: i+1 for i, team in enumerate(self.team_names)}
        
        # Initialize player data
        self.available_players = cheat_sheet.copy()
        self.available_players['Available'] = True
        self.available_players['Drafted_By'] = None
        self.available_players['Pick_Number'] = None
        self.available_players['Round'] = None
        
        # Draft state
        self.current_pick = 1
        self.current_round = 1
        self.draft_history = []
        
        # Team rosters (dict of team_name -> list of players)
        self.team_rosters = {team: [] for team in self.team_names}
        
        # Snake draft order
        self.draft_order = self._generate_snake_order()
        
    def set_draft_position(self, team_name, position):
        """Set a team's draft position (1-14)"""
        if 1 <= position <= self.num_teams:
            # Update position mapping
            old_position = self.draft_positions[team_name]
            
            # Find team currently at this position and swap
            for team, pos in self.draft_positions.items():
                if pos == position:
                    self.draft_positions[team] = old_position
                    break
                    
            self.draft_positions[team_name] = position
            
            # Regenerate draft order
            self.draft_order = self._generate_snake_order()
            print(f"✅ {team_name} moved to draft position {position}")
            return True
        else:
            print(f"❌ Invalid position {position}. Must be 1-{self.num_teams}")
            return False
    
    def _generate_snake_order(self):
        """Generate snake draft pick order based on current positions"""
        # Create position-to-team mapping
        position_to_team = {pos: team for team, pos in self.draft_positions.items()}
        
        order = []
        for round_num in range(1, self.roster_size + 1):
            if round_num % 2 == 1:  # Odd rounds: 1, 2, 3, ..., 14
                positions = list(range(1, self.num_teams + 1))
            else:  # Even rounds: 14, 13, 12, ..., 1
                positions = list(range(self.num_teams, 0, -1))
            
            for pick_in_round, position in enumerate(positions, 1):
                overall_pick = (round_num - 1) * self.num_teams + pick_in_round
                team = position_to_team[position]
                
                order.append({
                    'overall_pick': overall_pick,
                    'round': round_num,
                    'pick_in_round': pick_in_round,
                    'draft_position': position,
                    'team': team
                })
        return order
    
    def get_current_team(self):
        """Get team currently on the clock"""
        if self.current_pick <= len(self.draft_order):
            return self.draft_order[self.current_pick - 1]['team']
        return None
    
    def get_current_draft_position(self):
        """Get draft position currently on the clock"""
        if self.current_pick <= len(self.draft_order):
            return self.draft_order[self.current_pick - 1]['draft_position']
        return None
    
    def draft_player(self, player_name):
        """Draft a player to the current team (automatic assignment)"""
        if self.current_pick > len(self.draft_order):
            print("🏁 Draft is complete!")
            return False
            
        # Find player
        player_idx = self.available_players[self.available_players['Player'] == player_name].index
        if len(player_idx) == 0:
            print(f"❌ Player '{player_name}' not found or already drafted")
            return False
            
        player_idx = player_idx[0]
        
        # Get current team and pick info (automatic assignment)
        pick_info = self.draft_order[self.current_pick - 1]
        current_team = pick_info['team']
        draft_position = pick_info['draft_position']
        
        # Update player data
        self.available_players.loc[player_idx, 'Available'] = False
        self.available_players.loc[player_idx, 'Drafted_By'] = current_team
        self.available_players.loc[player_idx, 'Pick_Number'] = self.current_pick
        self.available_players.loc[player_idx, 'Round'] = pick_info['round']
        
        # Add to team roster
        player_data = self.available_players.loc[player_idx].to_dict()
        self.team_rosters[current_team].append(player_data)
        
        # Add to draft history
        self.draft_history.append({
            'pick': self.current_pick,
            'round': pick_info['round'],
            'pick_in_round': pick_info['pick_in_round'],
            'draft_position': draft_position,
            'team': current_team,
            'player': player_name,
            'position': player_data['Position'],
            'vbd': player_data['Custom_VBD'],
            'timestamp': datetime.now().strftime('%H:%M:%S')
        })
        
        print(f"✅ Pick {self.current_pick}: {current_team} (#{draft_position}) selects {player_name} ({player_data['Position']})")
        
        # Advance to next pick
        self.current_pick += 1
        if self.current_pick <= len(self.draft_order):
            self.current_round = self.draft_order[self.current_pick - 1]['round']
        
        return True
    
    def undo_last_pick(self):
        """Undo the last draft pick"""
        if not self.draft_history:
            print("❌ No picks to undo")
            return False
            
        last_pick = self.draft_history.pop()
        player_name = last_pick['player']
        team = last_pick['team']
        
        # Find and restore player
        player_idx = self.available_players[self.available_players['Player'] == player_name].index[0]
        self.available_players.loc[player_idx, 'Available'] = True
        self.available_players.loc[player_idx, 'Drafted_By'] = None
        self.available_players.loc[player_idx, 'Pick_Number'] = None
        self.available_players.loc[player_idx, 'Round'] = None
        
        # Remove from team roster
        self.team_rosters[team] = [p for p in self.team_rosters[team] if p['Player'] != player_name]
        
        # Revert pick counter
        self.current_pick -= 1
        if self.current_pick > 0:
            self.current_round = self.draft_order[self.current_pick - 1]['round']
        else:
            self.current_round = 1
        
        print(f"↩️ Undid pick: {player_name} returned to available players")
        return True
    
    def get_available_by_position(self, position=None, top_n=20):
        """Get available players, optionally filtered by position"""
        available = self.available_players[self.available_players['Available'] == True].copy()
        
        if position and position != 'ALL':
            available = available[available['Position'] == position]
        
        return available.head(top_n)[['Draft_Rank', 'Player', 'Position', 'Team', 'Bye', 'ECR', 'Custom_VBD']]
    
    def get_next_few_picks(self, num_picks=5):
        """Get the next few picks in the draft order"""
        next_picks = []
        for i in range(num_picks):
            pick_num = self.current_pick + i
            if pick_num <= len(self.draft_order):
                pick_info = self.draft_order[pick_num - 1]
                next_picks.append({
                    'pick': pick_num,
                    'round': pick_info['round'],
                    'team': pick_info['team'],
                    'draft_position': pick_info['draft_position']
                })
        return next_picks
    
    def save_draft_state(self, filename='auto_draft_state.pkl'):
        """Save current draft state"""
        state = {
            'available_players': self.available_players,
            'team_rosters': self.team_rosters,
            'draft_history': self.draft_history,
            'current_pick': self.current_pick,
            'current_round': self.current_round,
            'draft_positions': self.draft_positions
        }
        with open(filename, 'wb') as f:
            pickle.dump(state, f)
        print(f"💾 Draft state saved to {filename}")
    
    def load_draft_state(self, filename='auto_draft_state.pkl'):
        """Load draft state from file"""
        try:
            with open(filename, 'rb') as f:
                state = pickle.load(f)
            
            self.available_players = state['available_players']
            self.team_rosters = state['team_rosters']
            self.draft_history = state['draft_history']
            self.current_pick = state['current_pick']
            self.current_round = state['current_round']
            
            # Load draft positions if available
            if 'draft_positions' in state:
                self.draft_positions = state['draft_positions']
                self.draft_order = self._generate_snake_order()
            
            print(f"📂 Draft state loaded from {filename}")
            return True
        except FileNotFoundError:
            print(f"❌ File {filename} not found")
            return False

# Initialize auto draft tracker
auto_draft = AutoDraftTracker(config, cheat_sheet)
current_team = auto_draft.get_current_team()
current_pos = auto_draft.get_current_draft_position()
print(f"\n🏈 Auto-Draft initialized!")
print(f"⏰ Team on the clock: {current_team} (Draft Position #{current_pos})")


🏈 Auto-Draft initialized!
⏰ Team on the clock: Bam Bam Kam (Draft Position #1)


In [ ]:
# Draft Position Management Interface
class DraftPositionManager:
    def __init__(self, auto_draft_tracker):
        self.draft = auto_draft_tracker
        self.setup_widgets()
        
    def setup_widgets(self):
        # Position assignment widgets
        self.team_selector = widgets.Dropdown(
            options=self.draft.team_names,
            description='Team:',
            style={'description_width': 'initial'}
        )
        
        self.position_selector = widgets.IntSlider(
            value=1,
            min=1,
            max=self.draft.num_teams,
            description='Draft Position:',
            style={'description_width': 'initial'}
        )
        
        self.set_position_button = widgets.Button(
            description='Set Position',
            button_style='info'
        )
        
        self.position_output = widgets.Output()
        
        # Event handlers
        self.set_position_button.on_click(self.set_position)
        
        # Initial display
        self.refresh_display()
        
    def set_position(self, button):
        """Set a team's draft position"""
        with self.position_output:
            clear_output()
            team = self.team_selector.value
            position = self.position_selector.value
            
            success = self.draft.set_draft_position(team, position)
            if success:
                self.refresh_display()
    
    def refresh_display(self):
        """Refresh the draft position display"""
        with self.position_output:
            clear_output(wait=True)
            
            print("🎯 CURRENT DRAFT POSITIONS:\n")
            
            # Sort teams by draft position
            sorted_positions = sorted(self.draft.draft_positions.items(), key=lambda x: x[1])
            
            for team, position in sorted_positions:
                print(f"Position {position:2d}: {team}")
                
            print(f"\n⏰ Current Pick: {self.draft.current_pick}")
            current_team = self.draft.get_current_team()
            current_pos = self.draft.get_current_draft_position()
            if current_team:
                print(f"🎯 On the Clock: {current_team} (#{current_pos})")
    
    def display_interface(self):
        """Display the position management interface"""
        return widgets.VBox([
            widgets.HTML("<h4>🎯 Draft Position Management</h4>"),
            widgets.HBox([self.team_selector, self.position_selector, self.set_position_button]),
            self.position_output
        ])

# Create position manager
position_manager = DraftPositionManager(auto_draft)
display(position_manager.display_interface())

In [ ]:
# Streamlined Draft Interface
class StreamlinedDraftInterface:
    def __init__(self, auto_draft_tracker):
        self.draft = auto_draft_tracker
        self.setup_widgets()
        
    def setup_widgets(self):
        # Player selection widgets
        self.player_search = widgets.Text(
            placeholder='Search player name...',
            description='Search:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.position_filter = widgets.Dropdown(
            options=['ALL', 'QB', 'RB', 'WR', 'TE', 'DEF', 'K'],
            value='ALL',
            description='Position:',
            layout=widgets.Layout(width='150px')
        )
        
        self.player_dropdown = widgets.Dropdown(
            options=[],
            description='Select Player:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Action buttons
        self.draft_button = widgets.Button(
            description='🎯 DRAFT PLAYER',
            button_style='success',
            icon='check',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.undo_button = widgets.Button(
            description='↩️ UNDO',
            button_style='warning',
            layout=widgets.Layout(width='100px')
        )
        
        self.save_button = widgets.Button(
            description='💾 SAVE',
            button_style='info',
            layout=widgets.Layout(width='100px')
        )
        
        self.load_button = widgets.Button(
            description='📂 LOAD',
            button_style='info',
            layout=widgets.Layout(width='100px')
        )
        
        # Output areas
        self.status_output = widgets.Output(layout=widgets.Layout(height='100px'))
        self.draft_info_output = widgets.Output(layout=widgets.Layout(height='150px'))
        self.players_output = widgets.Output()
        self.rosters_output = widgets.Output()
        
        # Event handlers
        self.player_search.observe(self.update_player_list, names='value')
        self.position_filter.observe(self.update_player_list, names='value')
        self.draft_button.on_click(self.draft_player)
        self.undo_button.on_click(self.undo_pick)
        self.save_button.on_click(self.save_draft)
        self.load_button.on_click(self.load_draft)
        
        # Initial setup
        self.update_player_list()
        self.refresh_all_displays()
        
    def update_player_list(self, change=None):
        """Update available players dropdown"""
        available = self.draft.available_players[self.draft.available_players['Available'] == True].copy()
        
        # Apply position filter
        if self.position_filter.value != 'ALL':
            available = available[available['Position'] == self.position_filter.value]
        
        # Apply search filter
        if self.player_search.value:
            search_term = self.player_search.value.lower()
            available = available[available['Player'].str.lower().str.contains(search_term, na=False)]
        
        # Create dropdown options
        available = available.sort_values('Draft_Rank').head(30)
        options = [(f"{row['Draft_Rank']}. {row['Player']} ({row['Position']}, {row['Team']}) - VBD: {row['Custom_VBD']:.1f}", 
                   row['Player']) for _, row in available.iterrows()]
        
        self.player_dropdown.options = options
        if options:
            self.player_dropdown.value = options[0][1]
        
        self.refresh_players_display()
        
    def draft_player(self, button):
        """Handle draft button click - automatic team assignment"""
        if not self.player_dropdown.value:
            with self.status_output:
                clear_output()
                print("❌ No player selected")
            return
            
        with self.status_output:
            clear_output()
            success = self.draft.draft_player(self.player_dropdown.value)
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def undo_pick(self, button):
        """Handle undo button click"""
        with self.status_output:
            clear_output()
            success = self.draft.undo_last_pick()
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def save_draft(self, button):
        """Save draft state"""
        with self.status_output:
            clear_output()
            self.draft.save_draft_state()
            
    def load_draft(self, button):
        """Load draft state"""
        with self.status_output:
            clear_output()
            success = self.draft.load_draft_state()
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def refresh_draft_info(self):
        """Refresh current draft information"""
        with self.draft_info_output:
            clear_output()
            
            current_team = self.draft.get_current_team()
            current_pos = self.draft.get_current_draft_position()
            
            if current_team:
                print(f"🎯 Pick {self.draft.current_pick} (Round {self.draft.current_round})")
                print(f"⏰ ON THE CLOCK: {current_team} (Draft Position #{current_pos})")
                
                # Show next few picks
                next_picks = self.draft.get_next_few_picks(4)
                print("\n📅 UPCOMING PICKS:")
                for pick_info in next_picks[1:]:  # Skip current pick
                    print(f"  Pick {pick_info['pick']}: {pick_info['team']} (#{pick_info['draft_position']})")
            else:
                print("🏁 Draft Complete!")
                
    def refresh_players_display(self):
        """Refresh available players table"""
        with self.players_output:
            clear_output()
            
            print("📊 TOP AVAILABLE PLAYERS\n")
            
            # Available players table
            available = self.draft.get_available_by_position(self.position_filter.value, 12)
            if not available.empty:
                display(HTML(available.to_html(index=False, classes='table table-striped')))
            else:
                print("No available players match current filters")
                
    def refresh_rosters_display(self):
        """Refresh team rosters - compact view"""
        with self.rosters_output:
            clear_output()
            
            print("📋 TEAM ROSTERS (Recent Picks)\n")
            
            # Show recent picks for each team
            for i, team in enumerate(self.draft.team_names):
                roster = self.draft.team_rosters[team]
                draft_pos = self.draft.draft_positions[team]
                
                print(f"{team} (#{draft_pos}) - {len(roster)} picks:")
                
                if roster:
                    # Show last 3 picks
                    recent_picks = roster[-3:] if len(roster) > 3 else roster
                    for player in recent_picks:
                        round_pick = f"R{player['Round']}"
                        print(f"  {round_pick}: {player['Player']} ({player['Position']})")
                    
                    if len(roster) > 3:
                        print(f"  ... and {len(roster) - 3} more")
                else:
                    print("  No picks yet")
                    
                print()  # Empty line
                
    def refresh_all_displays(self):
        """Refresh all display areas"""
        self.refresh_draft_info()
        self.refresh_players_display()
        self.refresh_rosters_display()
        
    def display_interface(self):
        """Display the streamlined interface"""
        # Main controls
        search_row = widgets.HBox([self.player_search, self.position_filter])
        player_row = widgets.HBox([self.player_dropdown])
        action_row = widgets.HBox([self.draft_button, self.undo_button, self.save_button, self.load_button])
        
        controls = widgets.VBox([
            widgets.HTML("<h3>🏈 Streamlined Draft Interface</h3>"),
            search_row,
            player_row,
            action_row,
            self.status_output
        ])
        
        # Information panels
        info_tabs = widgets.Tab(children=[
            widgets.VBox([self.draft_info_output, self.players_output]),
            self.rosters_output
        ])
        info_tabs.set_title(0, '🎯 Current Pick & Available Players')
        info_tabs.set_title(1, '📋 Team Rosters')
        
        return widgets.VBox([controls, info_tabs])

# Create and display streamlined interface
interface = StreamlinedDraftInterface(auto_draft)
display(interface.display_interface())

In [6]:
# Quick Action Buttons for Testing
def quick_draft_round(num_picks=14):
    """Quick draft a full round for testing"""
    print(f"🚀 Quick drafting {num_picks} picks...\n")
    
    picks_made = 0
    for i in range(num_picks):
        if auto_draft.current_pick > len(auto_draft.draft_order):
            break
            
        # Get top available player
        available = auto_draft.get_available_by_position(top_n=1)
        if available.empty:
            break
            
        top_player = available.iloc[0]['Player']
        if auto_draft.draft_player(top_player):
            picks_made += 1
    
    # Refresh interface
    interface.update_player_list()
    interface.refresh_all_displays()
    print(f"\n✅ Quick draft complete! Made {picks_made} picks.")

def reset_draft():
    """Reset draft to beginning"""
    global auto_draft, interface
    auto_draft = AutoDraftTracker(config, cheat_sheet)
    interface = StreamlinedDraftInterface(auto_draft)
    print("🔄 Draft reset to beginning")
    display(interface.display_interface())

# Quick action buttons
quick_5 = widgets.Button(
    description='Quick 5 Picks', 
    button_style='warning',
    layout=widgets.Layout(width='120px')
)

quick_round = widgets.Button(
    description='Quick Round', 
    button_style='warning',
    layout=widgets.Layout(width='120px')
)

reset_button = widgets.Button(
    description='🔄 Reset Draft', 
    button_style='danger',
    layout=widgets.Layout(width='120px')
)

def quick_5_handler(button):
    quick_draft_round(5)
    
def quick_round_handler(button):
    quick_draft_round(14)
    
def reset_handler(button):
    reset_draft()
    
quick_5.on_click(quick_5_handler)
quick_round.on_click(quick_round_handler)
reset_button.on_click(reset_handler)

print("⚡ QUICK ACTIONS (for testing):")
display(widgets.HBox([quick_5, quick_round, reset_button]))

⚡ QUICK ACTIONS (for testing):


In [7]:
# Draft Visualization
def create_auto_draft_viz():
    """Create visualization for automatic draft"""
    if not auto_draft.draft_history:
        print("📊 Draft visualization will appear after picks are made")
        return
    
    # Prepare data
    draft_df = pd.DataFrame(auto_draft.draft_history)
    
    # Create draft board
    fig = go.Figure()
    
    # Position colors
    pos_colors = {
        'QB': '#FF6B6B',   # Red
        'RB': '#4ECDC4',   # Teal  
        'WR': '#45B7D1',   # Blue
        'TE': '#96CEB4',   # Green
        'DEF': '#FFEAA7',  # Yellow
        'K': '#DDA0DD'     # Plum
    }
    
    for _, pick in draft_df.iterrows():
        fig.add_trace(go.Scatter(
            x=[pick['draft_position']],
            y=[pick['round']],
            mode='markers+text',
            marker=dict(
                size=25,
                color=pos_colors.get(pick['position'], '#95A5A6'),
                line=dict(width=2, color='white')
            ),
            text=pick['position'],
            textposition="middle center",
            textfont=dict(size=9, color='white'),
            hovertemplate=(
                f"<b>Pick {pick['pick']}</b><br>"
                f"Round {pick['round']}, Position {pick['draft_position']}<br>"
                f"<b>{pick['player']}</b><br>"
                f"{pick['position']} - {pick['team']}<br>"
                f"VBD: {pick['vbd']:.1f}<extra></extra>"
            ),
            showlegend=False
        ))
    
    fig.update_layout(
        title=f"🏈 Draft Board - {len(auto_draft.draft_history)} Picks Made",
        xaxis=dict(
            title="Draft Position",
            tickmode='linear',
            tick0=1,
            dtick=1,
            range=[0.5, auto_draft.num_teams + 0.5]
        ),
        yaxis=dict(
            title="Round",
            tickmode='linear', 
            tick0=1,
            dtick=1,
            autorange='reversed'
        ),
        height=500,
        plot_bgcolor='white'
    )
    
    fig.show()

# Visualization button
viz_button = widgets.Button(
    description='📊 Show Draft Board',
    button_style='primary',
    icon='chart-bar'
)

viz_button.on_click(lambda b: create_auto_draft_viz())
display(viz_button)

Button(button_style='primary', description='📊 Show Draft Board', icon='chart-bar', style=ButtonStyle())

## 🎯 How to Use the Auto-Draft System

### 🔧 Setup (One-time):
1. **Set Draft Positions**: Use the Draft Position Management section to assign teams to positions 1-14
2. **Verify Order**: Check that teams are in the correct draft order

### 🏈 During the Draft:
1. **Current Pick Display**: Always shows which team/position is on the clock
2. **Select Player**: 
   - Search by name or filter by position
   - Choose from the dropdown of top available players
3. **Draft**: Click "🎯 DRAFT PLAYER" - **no team selection needed!**
4. **Automatic Assignment**: Player goes to the current team automatically
5. **Next Pick**: System advances to next team in snake order

### ✨ Key Benefits:
- **No Manual Team Selection**: Just pick the player, system handles the rest
- **Automatic Snake Order**: Handles complex round reversals perfectly
- **Clear Pick Info**: Always know who's drafting and what position they're in
- **Upcoming Picks**: See the next 4 teams in order
- **Position Tracking**: Each team's draft position clearly displayed

### 🔄 Advanced Features:
- **Undo Function**: Mistakes can be reversed instantly
- **Save/Load**: Preserve draft progress
- **Quick Actions**: Test with simulated picks
- **Visual Draft Board**: See picks by position and round

### 📊 Information Views:
- **Tab 1**: Current pick info + available players
- **Tab 2**: Team rosters with recent picks

**Perfect for live drafts where you just want to focus on player selection! 🏆**